In [1]:

import pandas as pd
from itertools import groupby

In [2]:
ge_term_date = { 11: {'start': '21 March 2004', 'end': '13 February 2008'},
 12: {'start': '8 March 2008', 'end': '3 April 2013'},
 13: {'start': '5 May 2013', 'end': '7 April 2018'},
 14: {'start': '9 May 2018', 'end': '10 October 2022'},
 15: {'start': '19 November 2022', 'end': None}}

In [3]:
def are_consecutive(a):
    return sorted(a) == list(range(min(a), max(a) + 1))

def split_consec(lst):
    out = []
    for _, g in groupby(enumerate(lst), lambda x: x[0] - x[1]):
        out.append([v for _, v in g])
    return out

def gen_consec(i):
    if i["consecutive"]:
        return [i["vol"]]
    else:
        return split_consec(i["vol"])

def gen_date_set(i):
    _start = []
    _end = []
    for period in i["vol_set"]:
        _start.append(ge_term_date[min(period)]["start"])
        _end.append(ge_term_date[max(period)]["end"])
    return pd.Series([_start, _end], index=["start_date","end_date"])

In [4]:
# urls = ["https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_1st_Malayan_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_2nd_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_3rd_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_4th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_5th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_6th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_7th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_8th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_9th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_10th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_11th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_12th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_13th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_14th_Malaysian_Parliament",
# "https://en.wikipedia.org/wiki/Members_of_the_Dewan_Rakyat,_15th_Malaysian_Parliament",
# ]
# ge_term_date = {}
# for vol, url in enumerate(urls):
#     term_start, term_end = aaa, bbb = re.sub(r"\[.?\]","",pd.read_html(url)[0].iloc[6,1]).split("\xa0– ")
#     ge_term_date[vol + 1] = {"start": term_start, "end": None if term_end == "present" else term_end}
# ge_term_date

In [5]:
dn_tempoh = pd.read_csv("DN - dn_w_author_id.csv")
dn_tempoh["tempoh_mula"] = pd.to_datetime(dn_tempoh["tempoh_mula"])
dn_tempoh["tempoh_tamat"] = pd.to_datetime(dn_tempoh["tempoh_tamat"])
dn_tempoh = dn_tempoh.drop_duplicates()
dn_tempoh_min = dn_tempoh[["author_id","vol", "tempoh_mula", "tempoh_tamat"]].drop_duplicates()
dn_tempoh_min_copy = dn_tempoh_min.copy()
dn_tempoh_min = pd.merge(dn_tempoh_min[["author_id","vol"]], dn_tempoh_min_copy.groupby(["author_id","vol"])["tempoh_mula"].min().reset_index(), on=["author_id","vol"])
dn_tempoh_min = pd.merge(dn_tempoh_min[["author_id","vol","tempoh_mula"]], dn_tempoh_min_copy.groupby(["author_id","vol"])["tempoh_tamat"].max().reset_index(), on=["author_id","vol"])

dn_tempoh_min = dn_tempoh_min.drop_duplicates()

In [6]:
def gen_date_set_dn(i, raw_data=dn_tempoh_min):
    _start = []
    _end = []
    author_id = i["author_id"]
    author_data = raw_data[raw_data["author_id"]==author_id]
    for period in i["vol_set"]:
        _start.append(author_data[author_data["vol"] == min(period)]["tempoh_mula"].iloc[0])
        _end.append(author_data[author_data["vol"] == max(period)]["tempoh_tamat"].iloc[0])
    return pd.Series([_start, _end], index=["start_date","end_date"])

pooled_dn = dn_tempoh_min[["author_id", "vol", "tempoh_mula", "tempoh_tamat"]].sort_values(by="vol", ascending=True).groupby("author_id")["vol"].apply(list).reset_index()
# dn_tempoh_min["consecutive"] = 
pooled_dn["consecutive"] = pooled_dn["vol"].apply(are_consecutive)
pooled_dn["vol_set"] = pooled_dn[["vol","consecutive"]].apply(gen_consec,axis=1)
pooled_dn[["start_date","end_date"]] = pooled_dn.apply(gen_date_set_dn, axis=1)
pooled_dn = pooled_dn.explode(["vol_set","start_date", "end_date"])


dn_tempoh_remap = pd.merge(dn_tempoh[['author_id', 'vol', 'nama']].drop_duplicates(), dn_tempoh_min, on=["author_id","vol"])
dn_tempoh_remap.columns = ['author_id', 'vol', 'name', 'tempoh_mula', 'tempoh_tamat']
dn_tempoh_remap["name"] = dn_tempoh_remap["name"].str.upper()
dn_tempoh_remap[dn_tempoh_remap["vol"].isin([15])]["author_id"].nunique()
dn_tempoh_remap[dn_tempoh_remap["vol"].isin([12,13,14,15])]["vol"].value_counts()
author_history = pd.read_csv("author_history.csv").drop_duplicates()
test_merge = pd.merge(author_history, dn_tempoh_remap[["author_id", "name","vol",'tempoh_mula', 'tempoh_tamat']], on=["name", "vol"], how="left")
dn_author_id_mapping = test_merge[~test_merge["author_id_y"].isna()][["new_uid","author_id_y"]].drop_duplicates().set_index("author_id_y").to_dict()["new_uid"]

In [7]:
dn_tempoh_min_filt = dn_tempoh_min[dn_tempoh_min["vol"].isin([12,13,14,15])]
dn_tempoh_min_filt["new_uid"] = dn_tempoh_min_filt["author_id"].map(dn_author_id_mapping)
author_hist_merge = pd.merge(author_history, dn_tempoh_min_filt[['vol', 'tempoh_mula', 'tempoh_tamat', 'new_uid']], "left")
author_hist_dn = author_hist_merge[~author_hist_merge["tempoh_mula"].isna()]
dr_auth_hist = author_hist_merge[author_hist_merge["tempoh_mula"].isna()]["new_uid"].values
author_hist_dn = author_hist_dn[['vol', 'name', 'birth_year', 'sex', 'ethnicity', 'parti', 'negeri',
       'kawasan', 'kod_kawasan', 'jawatan_kabinet', 'jawatan_parlimen',
       'new_uid', 'tempoh_mula', 'tempoh_tamat']]
author_hist_dn.columns = ['vol', 'author', 'birth_year', 'sex', 'ethnicity', 'party', 'negeri',
       'area', 'kod_kawasan', 'exec_posts', 'service_posts',
       'author_id', 'start_date', 'end_date']


/var/folders/7q/vk27jw6d7_95l168mby_n6zc0000gn/T/ipykernel_33842/1022689298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dn_tempoh_min_filt["new_uid"] = dn_tempoh_min_filt["author_id"].map(dn_author_id_mapping)


In [9]:
ge_term_date_df = pd.DataFrame.from_dict(ge_term_date).T.reset_index(names=["vol"])
merged_term = pd.merge(author_history[author_history["new_uid"].isin(dr_auth_hist)], ge_term_date_df, "left")
test = merged_term.sort_values(by="vol", ascending=True)[["new_uid", "vol"]].groupby(by="new_uid")["vol"].apply(list).reset_index()
test["consecutive"] = test["vol"].apply(are_consecutive)
test["vol_set"]=test[["vol", "consecutive"]].apply(gen_consec,axis=1)
test[["start_date","end_date"]] = test[["vol_set"]].apply(gen_date_set,axis=1)
test2 = test[["new_uid", "start_date", "end_date"]].explode(["start_date", "end_date"])

merged_term_fil = merged_term[['vol', 'name', 'birth_year', 'sex', 'ethnicity', 'parti', 'negeri',
       'kawasan', 'kod_kawasan', 'jawatan_kabinet', 'jawatan_parlimen', 'new_uid', 'start', 'end']]
merged_term_fil.columns = ['vol', 'author', 'birth_year', 'sex', 'ethnicity', 'party', 'negeri',
       'area', 'kod_kawasan', 'exec_posts', 'service_posts',
       'author_id', 'start_date', 'end_date']
merged_term_fil["start_date"] = pd.to_datetime(merged_term_fil["start_date"])
merged_term_fil["end_date"] = pd.to_datetime(merged_term_fil["end_date"])


final_author_history = pd.concat([merged_term_fil, author_hist_dn])
final_author_history["service_posts"] = final_author_history["service_posts"].replace("TIADA", None)
final_author_history["exec_posts"] = final_author_history["exec_posts"].replace("TIADA", None)
final_author_history.to_parquet("author_history_with_term.parquet", index=False)
final_author_history.to_csv("author_history_with_term.csv", index=False)

/var/folders/7q/vk27jw6d7_95l168mby_n6zc0000gn/T/ipykernel_33842/3842822722.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_term_fil["start_date"] = pd.to_datetime(merged_term_fil["start_date"])
/var/folders/7q/vk27jw6d7_95l168mby_n6zc0000gn/T/ipykernel_33842/3842822722.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_term_fil["end_date"] = pd.to_datetime(merged_term_fil["end_date"])


In [10]:
author = pd.read_csv("author.csv")
pooled_dn["author_id"] = pooled_dn["author_id"].map(dn_author_id_mapping)
dn_author = pd.merge(author, pooled_dn[['author_id','start_date', 'end_date',]], how="left", on=["author_id"])
dn_author_term = dn_author[~dn_author["start_date"].isna()]
# author = pd.read_csv("author.csv")
author_with_term = pd.merge(author[author["author_id"].isin(dn_author[dn_author["start_date"].isna()]["author_id"].values)], test2, "left", left_on="author_id", right_on="new_uid")
author_with_term["start_date"] = pd.to_datetime(author_with_term["start_date"])
author_with_term["end_date"] = pd.to_datetime(author_with_term["end_date"])
author_with_term = author_with_term.drop("new_uid", axis=1)

author_term_export = pd.concat([dn_author_term, author_with_term])
author_term_export.to_parquet("author_with_term.parquet", index=False)
author_term_export.to_csv("author_with_term.csv", index=False)